# DiSignAtlas

In [44]:
"""DiSignAtlas

This Notebook is used for exploring the DiSignAtlas dataset.

Strcuture:
    a. Disease Mapping
"""

'DiSignAtlas\n\nThis Notebook is used for exploring the DiSignAtlas dataset.\n\nStrcuture:\n    a. Disease Mapping\n'

## a. Disease Mapping

In [49]:
"""Disease Mapping
Here we are interested in Mapping the diseases to the corresponding MeSH terms.

Structure:
    1. Imports, Variables and Functions
    2. Load Data
    3. Disease Mapping
"""

# 1. Imports, Variables and Functions
# imports
import pandas as pd, numpy as np, os, sys, re, json, pickle, time, datetime, random
import requests
import xml.etree.ElementTree as ET
import logging
import json
from tqdm import tqdm
from bs4 import BeautifulSoup

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
# variables
data_path = os.path.join("..", "data", "DiSignAtlas", "dis_info_datasets")
mesh_file_path = "../data/MeSH/desc2023.xml"
output_path = os.path.join(
    "..", "results", "files", "DiSignAtlas", "disease_mapping.nlp.csv"
)
# functions


def retrieve_external_links(ids):
    base_url = "http://www.inbirg.com/disignatlas/detail/"
    results = {}

    for id in ids:
        url = f"{base_url}{id}"
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            print(response.content)
            external_links_section = soup.find("div", {"id": "external_links"})
            print(external_links_section)
            if external_links_section:
                links = external_links_section.find_all("a")
                results[id] = [
                    link.get("href")
                    for link in links
                    if "MeSH" in link.text or "DO" in link.text
                ]
            else:
                results[id] = "No external links section found"
        else:
            results[id] = "Failed to retrieve data"

    return results


# 2. Load Data
# load disease info
# Example usage with a list of IDs
example_ids = ["DSA00047"]
retrieve_external_links(example_ids)

# load disease mapping


# 3. Disease Mapping
# map diseases to MeSH terms

# Discarded because it takes too long to run ! ! !
# disease_mapping = list()

# for disease in tqdm(diseases):
#     best_match, best_score = find_best_fuzzy_match(disease, disease_mesh_terms)
#     disease_mapping.append([disease, best_match, best_score])


# Load the 'en_core_web_md' model

# Perform disease mapping


# save disease mapping

b'<!DOCTYPE html>\n<html lang="en">\n\t<head>\n\t\t<meta charset="UTF-8">\n\t\t<title>Detail</title>\n\t\t<!-- Bootstrap CSS -->\n<link rel="stylesheet" href="/diseasedb_static/cdn/css/bootstrap-4.3.1.min.css">\n<script src="/diseasedb_static/cdn/js/jquery-3.2.1.min.js"></script>\n<!-- <script type="text/javascript" charset="utf8" src="http://code.jquery.com/jquery-1.10.2.min.js"></script> -->\n<!--jquery1_test-->\n<!-- <script type="text/javascript" charset="utf8" src="http://code.jquery.com/jquery-1.10.2.min.js"></script> -->\n<script src="/diseasedb_static/cdn/js/popper-1.15.0.min.js"></script>\n<script src="/diseasedb_static/cdn/js/bootstrap-4.3.1.min.js"></script>\n<!-- bootstrap-select -->\n<link rel="stylesheet" href="/diseasedb_static/cdn/css/bootstrap-select-1.13.14.min.css">\n<!-- <script src="https://cdn.jsdelivr.net/npm/bootstrap-select@1.13.14/dist/js/i18n/defaults-*.min.js"></script> -->\n<!-- cytoscape -->\n<!-- <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.co

{'DSA00047': 'No external links section found'}

In [46]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 16.6 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


## b. DiSignAtlas get MeSH terms from DataSets

In [47]:
"""DiSginAtlas DataSets

The objective here is to retrieve the datasets from DiSignAtlas.

Structure:
    1. Imports, Variables and Functions
    2. Load Data
    3. Retrieve DataSets
"""

# 1. Imports, Variables and Functions
# imports
import pandas as pd, numpy as np, os, sys, re, json, pickle, time, datetime, random
import requests
import xml.etree.ElementTree as ET
import time, re
from Bio import Entrez
import logging
import pandas as pd
import json
import logging

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

# variables
dataset_info_path = os.path.join("..", "data", "DiSignAtlas", "dis_info_datasets")
Entrez.email = "dylandaltonsub@gmail.com"

# functions

# 1. Imports, Variables, Functions
# imports
import requests
import xml.etree.ElementTree as ET
import time, re
from Bio import Entrez
import logging
import pandas as pd
import json

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
# variables
base_url = "http://www.ilincs.org/api"


# functions


def fetch_mesh_terms_from_pubmed(pmid, max_retries=10, retry_delay=5):
    """Fetch MeSH terms for a given PubMed ID, with retries."""
    if not pmid:
        return []

    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    params = {"db": "pubmed", "id": pmid, "retmode": "xml"}
    attempts = 0

    while attempts < max_retries:
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            # Parse the XML response to extract MeSH terms and tree numbers
            root = ET.fromstring(response.text)
            mesh_terms = [
                descriptor.findtext("DescriptorName")
                for descriptor in root.findall(".//MeshHeading")
            ]

            mesh_tree_numbers = []
            for descriptor in root.findall(".//MeshHeading"):
                descriptor_ui = descriptor.find("DescriptorName").get("UI")
                tree_numbers = root.findall(
                    f".//DescriptorRecord[DescriptorUI='{descriptor_ui}']/TreeNumberList/TreeNumber"
                )
                mesh_tree_numbers.extend(
                    [tree_number.text for tree_number in tree_numbers]
                )

            return mesh_terms, mesh_tree_numbers

        else:
            print(
                f"Attempt {attempts + 1} failed: Error fetching MeSH terms for PMID {pmid}: {response.text}"
            )
            time.sleep(retry_delay)
            attempts += 1

    print("Max retries reached. Failed to fetch MeSH terms.")
    return []


# 2. Load Data
# retrieve disease datasets
df_dataset_info = pd.read_csv(dataset_info_path)
dataset_ids = df_dataset_info["accession"].unique().tolist()

# 3. Retrieve MeSH Terms

# Retrieve pmid
pmids = list()
for dataset_id in tqdm(dataset_ids):
    pmid = get_pmid_from_geo_via_eutils(dataset_id)
    pmids.append(pmid)

  1%|          | 34/5059 [01:03<2:36:36,  1.87s/it]


KeyboardInterrupt: 

In [ ]:
from Bio import Entrez

# Set your email address here
Entrez.email = "your_email@example.com"


def get_pmid_from_geo_series(geo_series_id):
    try:
        # Search in GEO database for the given series accession ID
        handle = Entrez.esearch(db="geo", term=geo_series_id)
        record = Entrez.read(handle)
        handle.close()

        # Extract the GEO series ID list
        geo_id_list = record["IdList"]
        if not geo_id_list:
            print("No GEO record found for the given GEO series ID.")
            return None

        # Use elink to establish links between GEO and PubMed databases
        handle = Entrez.elink(dbfrom="geo", db="pubmed", id=geo_id_list[0])
        record = Entrez.read(handle)
        handle.close()

        # Check if LinkSetDb is present
        if "LinkSetDb" in record[0] and len(record[0]["LinkSetDb"]) > 0:
            # Extract and return all PMIDs
            pmids = [link["Id"] for link in record[0]["LinkSetDb"][0]["Link"]]
            return pmids
        else:
            return None

    except Exception as e:
        print(f"An error occurred: {e}")
        return None


get_pmid_from_geo_series("GSE11352")

An error occurred: Invalid db name specified: geo


In [ ]:
for dataset_id in tqdm(dataset_ids):
    handle = Entrez.esearch(db="pmc", term=dataset_id)
    print(Entrez.read(handle)["IdList"])

  0%|          | 1/5059 [00:00<48:25,  1.74it/s]

['9925742']


  0%|          | 2/5059 [00:01<46:27,  1.81it/s]

[]


  0%|          | 3/5059 [00:01<46:10,  1.83it/s]

...

 16%|█▌        | 800/5059 [08:15<39:43,  1.79it/s]

['10556489', '9714191', '9585216', '9530920', '8863469', '8922093', '8810074', '8439286', '7694566', '6788726']


 16%|█▌        | 801/5059 [08:15<43:54,  1.62it/s]

[]


KeyboardInterrupt: 

In [53]:
df_dataset_info[df_dataset_info["organism"] == "Homo sapiens"].shape
# df_dataset_info.shape

(7194, 12)

In [56]:
df_dataset_info

,dsaid,accession,platform,deg_count,disease,diseaseid,tissue,data_source,library_strategy,organism,control_case_sample_count,definition
0,DSA00001,GSE224398,GPL21103,1000,Alzheimer's Disease,C0002395,Hippocampus,GEO,scRNA-Seq,Mus musculus,1|1,DO:An Alzheimer's disease that has_material_ba...
1,DSA00002,GSE224398,GPL21103,1000,Alzheimer's Disease,C0002395,Hippocampus,GEO,scRNA-Seq,Mus musculus,1|1,DO:An Alzheimer's disease that has_material_ba...
2,DSA00003,GSE224398,GPL21103,1000,Alzheimer's Disease,C0002395,Hippocampus,GEO,scRNA-Seq,Mus musculus,1|1,DO:An Alzheimer's disease that has_material_ba...
3,DSA00004,GSE224022,GPL16791,1000,Retinoblastoma,C0035335,Retina,GEO,RNA-Seq,Homo sapiens,4|5,DO:A retinal cell cancer and malignant neoplas...
4,DSA00005,GSE126342,GPL11154,1000,Congenital Myotonic Dystrophy,C0410226,Skeletal muscle,GEO,RNA-Seq,Homo sapiens,9|11,MONDO:An inherited progressive disorder affect...
...,...,...,...,...,...,...,...,...,...,...,...,...
10301,DSA10302,GSE6008,GPL96,1000,Ovarian Tumor,C1140680,Ovary,GEO,Microarray,Homo sapiens,4|41,DO:A female reproductive organ cancer that is ...
10302,DSA10303,GSE6280,GPL96,758,Kidney Tumor,C0022665,Kidney,GEO,Microarray,Homo sapiens,6|14,DO:A urinary system cancer that is located_in ...
10303,DSA10304,GSE6280,GPL97,337,Kidney Tumor,C0022665,Kidney,GEO,Microarray,Homo sapiens,6|14,DO:A urinary system cancer that is located_in ...
10304,DSA10305,GSE6344,GPL96,1000,Clear Cell Ependymoma,C1384403,Kidney,GEO,Microarray,Homo sapiens,5|5,"EFO:A WHO grade II, slow growing tumor of chil..."


In [66]:
from Bio import Entrez


def translate_medgen_to_mesh(concept_id):
    Entrez.email = "example@example.com"  # Replace with your email address
    database = "medgen"
    term = concept_id

    # Search for the concept ID in the MedGen database
    search_handle = Entrez.esearch(db=database, term=term)
    search_results = Entrez.read(search_handle)
    search_handle.close()

    # Fetch the detailed record for the first UID in the search results
    uid = search_results["IdList"][0]
    fetch_handle = Entrez.efetch(db=database, id=uid, retmode="xml")
    data = Entrez.read(fetch_handle)
    fetch_handle.close()

    print(data)

    # Extract the MeSH term from the data
    # The path to the MeSH term in the data structure needs to be determined
    mesh_term = "Path to MeSH term in data"  # Placeholder, needs actual path

    return mesh_term


# Example usage
concept_id = "C0002395"
mesh_term = translate_medgen_to_mesh(concept_id)
mesh_term

['1853']


'Path to MeSH term in data'

In [68]:
search_handle = Entrez.esearch(db="medgen", term="C0002395")
search_results = Entrez.read(search_handle)

print(search_results)

# Fetch the detailed record for the UID
fetch_handle = Entrez.efetch(db=database, id=uid, retmode="xml")
data = Entrez.read(fetch_handle)
fetch_handle.close()

{'Count': '1', 'RetMax': '1', 'RetStart': '0', 'IdList': ['1853'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'C0002395[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'C0002395[All Fields]'}


In [61]:
def fetch_mesh_term_from_uid(uid):
    fetch_handle = Entrez.efetch(db="medgen", id=uid, retmode="xml")
    data = Entrez.read(fetch_handle)
    fetch_handle.close()

    print(data)
    # Extracting the MeSH term requires navigating the data structure
    # The exact path will depend on how the data is structured
    mesh_term = "Path to MeSH term in data"  # Placeholder, needs actual path

    return mesh_term


# The UID obtained from the previous step
medgen_uid = "1853"
mesh_term = fetch_mesh_term_from_uid(medgen_uid)
print("MeSH Term:", mesh_term)

['1853']
MeSH Term: Path to MeSH term in data


In [65]:
Entrez.read(Entrez.efetch(db="medgen", id="C0002395"))

['2395']